<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-VGG16/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**

## Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Import datatset

In [2]:
! pip install bing_image_downloader

In [3]:
from bing_image_downloader import downloader

In [ ]:
# Query
search_queries=[
    'Nicole Kidman',
    'Hugh Jackman',
    'Mel Gibson',
    'Russell Crowe',
    'Liam Hemsworth',
    'Cathy Freeman',
    'Margot Robbie'
]

dataset_path = 'dataset'

# Imgae downloder function
def donwloader(query, n_sample, dataset_name):
  downloader.download(query, limit=n_sample, output_dir=os.path.join(dataset_path, dataset_name), adult_filter_off=True, force_replace=False, timeout=60)

for query in search_queries:
  donwloader(query, n_sample=20, dataset_name='train')
  donwloader(query, n_sample=10, dataset_name='val')
  donwloader(query, n_sample=5, dataset_name='test')

In [5]:
dataset_path='/content/dataset'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

In [6]:
nr_files = nr_train_files = nr_val_files = nr_test_files = 0

for root, dirc, files in os.walk(dataset_path):
  nr_files += len(files)
print("#files: ", nr_files)
for root, dirc, files in os.walk(train_path):
  nr_train_files += len(files)
print("#train_files: ", nr_train_files)
for root, dirc, files in os.walk(val_path):
  nr_val_files += len(files)
print("#val_files: ", nr_val_files)
for root, dirc, files in os.walk(test_path):
  nr_test_files += len(files)
print("#test_files: ", nr_test_files)

#files:  185
#train_files:  100
#val_files:  50
#test_files:  35


In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [37]:
batch_size = 32
lr = 0.001
epoch = 10

# Define Image data generator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = train_datagen.flow_from_directory(
    val_path,
    target_size = (224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = train_datagen.flow_from_directory(
    test_path,
    target_size = (224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 100 images belonging to 7 classes.
Found 50 images belonging to 7 classes.
Found 35 images belonging to 7 classes.


In [38]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Frezz the layers exept the last Conv. layer
for layer in base_model.layers[:-4]:
  layer.trainable=False

# Add the customized classifier
block_5_fatten = Flatten()(base_model.output)
block_5_dense = Dense(256, activation='relu')(block_5_fatten)
block_5_drop_out = Dropout(0.6)(block_5_dense)

predictions = Dense(len(search_queries), activation='softmax')(block_5_drop_out)

# Build a new model
new_model = Model(inputs=base_model.output, outputs=predictions)
new_model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

## Train the New_Model

In [39]:
steps_per_epoch = round(train_generator.samples/batch_size)
validation_steps = round(val_generator.samples/batch_size)

In [40]:
history = new_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epoch,
    validation_data=val_generator,
    validation_steps=validation_steps)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node model_4/dense_10/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pytho